In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import numpy as np

In [16]:
num_epochs = 5 
num_classes = 10 
batch_size = 100 
learning_rate = 0.001

In [17]:
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

MNIST_train = torchvision.datasets.MNIST('./', download=True, train=True,  transform=trans)
MNIST_test = torchvision.datasets.MNIST('./', download=True, train=False,  transform=trans)

In [18]:
train_loader = DataLoader(dataset=MNIST_train, batch_size=batch_size,shuffle=True) 
test_loader = DataLoader(dataset=MNIST_test, batch_size=batch_size, shuffle=False)

In [19]:
class ConvNet(nn.Module): 
  def __init__(self): 
         super(ConvNet, self).__init__() 
         self.layer1 = nn.Sequential( nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2), 
            nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2)) 
         self.layer2 = nn.Sequential( nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2), 
            nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2)) 
         self.drop_out = nn.Dropout() 
         self.fc1 = nn.Linear(7 * 7 * 64, 1000) 
         self.fc2 = nn.Linear(1000, 10)
  def forward(self, x): 
     out = self.layer1(x) 
     out = self.layer2(out) 
     out = out.reshape(out.size(0), -1) 
     out = self.drop_out(out) 
     out = self.fc1(out) 
     out = self.fc2(out) 
     return out


In [20]:
model = ConvNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [21]:
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Прямой запуск
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Обратное распространение и оптимизатор
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Отслеживание точности
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/5], Step [100/600], Loss: 0.1907, Accuracy: 95.00%
Epoch [1/5], Step [200/600], Loss: 0.1327, Accuracy: 96.00%
Epoch [1/5], Step [300/600], Loss: 0.0893, Accuracy: 96.00%
Epoch [1/5], Step [400/600], Loss: 0.0832, Accuracy: 98.00%
Epoch [1/5], Step [500/600], Loss: 0.1445, Accuracy: 95.00%
Epoch [1/5], Step [600/600], Loss: 0.0668, Accuracy: 98.00%
Epoch [2/5], Step [100/600], Loss: 0.0062, Accuracy: 100.00%
Epoch [2/5], Step [200/600], Loss: 0.0550, Accuracy: 98.00%
Epoch [2/5], Step [300/600], Loss: 0.0339, Accuracy: 99.00%
Epoch [2/5], Step [400/600], Loss: 0.0851, Accuracy: 98.00%
Epoch [2/5], Step [500/600], Loss: 0.0750, Accuracy: 97.00%
Epoch [2/5], Step [600/600], Loss: 0.1233, Accuracy: 95.00%
Epoch [3/5], Step [100/600], Loss: 0.0154, Accuracy: 99.00%
Epoch [3/5], Step [200/600], Loss: 0.0397, Accuracy: 98.00%
Epoch [3/5], Step [300/600], Loss: 0.1387, Accuracy: 97.00%
Epoch [3/5], Step [400/600], Loss: 0.1387, Accuracy: 97.00%
Epoch [3/5], Step [500/600], Loss: 0.02

In [22]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format((correct / total) * 100))


Test Accuracy of the model on the 10000 test images: 98.97 %
